# 生物医学统计概论  单元3  @林关宁

## Dependent (paired) samples - Paired t-interval

This test is used when the samples are dependent; that is, when there is only one sample that has been tested twice (repeated measures) or when there are two samples that have been matched or "paired" 

Confidence interval of difference in means assuming paired samples can be calculated as follows:

$$ \text{C.I.}_{\Delta \text{mean}}: \quad \bar{d} \pm (t_{1-\frac{\alpha}{2}, df} \times \frac{s_d}{\sqrt{n}})  \tag{1}  $$

where

$\bar{d}$: average of sample differences

$α$: significance level

$n$: number of samples

$df$: degrees of freedom

$s_d$: standard deviation of sample differences

$t$: t-score. depends on α and degrees of freedom n−1

In [6]:
from scipy import stats
import numpy as np

例子：

一所学校制定了一个辅导计划来提高高中生的SAT（美国高考）成绩。学校要求学生在辅导前后进行测试，并检查辅导是否对学生的SAT成绩有显著影响。由于受试者是与自己进行比较的，而不是与其他人进行比较，因此培训前后的测量结果并不独立。.

To compute dependent t-interval, we compute differences of test scores before & after tutoring:

| Student# | $X_1$ | $X_2$ | $(X_1-X_2)$ |
| ----- | ----- | ----- | ----- |
| 1 | 1480 | 1510 | -30 |
| 1 | 1280 | 1460 | -180 |
| 1 | 890 | 1320 | -430 |
| 1 | 340 | 700 | -360 |
| 1 | 1550 | 1550 | 0 |
| 1 | 1230 | 1420 | -190 |
| 1 | 1010 | 1340 | -330 |
| 1 | 1590 | 1570 | 20 |
| 1 | 1390 | 1500 | -110 |
| 1 | 980 | 1300 | -320 |

We find $\bar{d}$ = -193.0, and $s_d$ = 161.7. These values are plugged into eq (1). Degrees of freedom $df$ for dependent t-interval can be computed with:

$$ df = n - 1 \tag{2} $$

In [7]:
x1 = np.array([1480, 1280, 890, 340, 1550, 1230, 1010, 1590, 1390, 980])
x2 = np.array([1510, 1460, 1320, 700, 1550, 1420, 1340, 1570, 1500, 1300])

alpha = 0.05                        # significance level = 5%
d_bar = np.mean(x1 - x2)            # average of sample differences
s_d = np.std(x1 - x2, ddof=1)       # sample standard deviation of sample differences
n = len(x1)                         # sample size
df = n - 1                          # degrees of freedom
t = stats.t.ppf(1 - alpha/2, df)    # t-critical value for 95% CI

lower = d_bar - t * s_d / np.sqrt(n)
upper = d_bar + t * s_d / np.sqrt(n)

print(lower, upper)

-308.64567899681356 -77.35432100318641


## Independent (unpaired) samples, equal variance - student's t-interval

When you have a reason to believe that samples have nearly equal variances, you can use student's t-test to check if difference in means are significantly different. Note that student's t-test works pretty well even with unequal variances as long as sample sample sizes are equal or nearly equal, and sample sizes are not tiny.

$$ \text{C.I.}_{\Delta \text{mean}}: \quad (\mu_{1}- \mu_{2}) \pm (t_{1-\frac{\alpha}{2},df} \times s_p\sqrt{\frac{1}{n_1}+\frac{1}{n_2}})\,, \quad s_p = \sqrt{\frac{(n_1-1)s_{1}^2 + (n_2-1)s_{2}^2}{n_1+n_2-2}} \tag{3}  $$

where

$μ$: sample mean

$α$: significance level

$n$: number of samples

$df$: degrees of freedom

$s_p$: pooled standard deviation

$s$: sample standard deviation

$t$: t-score. depends on α and degrees of freedom $n−1$

The formula for the pooled standard deviation $s_p$ looks a bit overwhelming, but its just an weighted average standard deviation of two samples, with bias correction factor $n_i−1$ for each sample. Recall that student's t-test assumes equal variances of two samples. You calculate what is assumed to be the common variance (=pooled variance, $s_{p}^2$) by computing the weighted average from each sample's variance.

In eq (2), t-score depends on significance level $α$ and degrees of freedom $df$. In student's t-test, which assumes equal variance:

$ df = n_1 + n_2 -2 \tag{4} $

#### Pythonic Tip: Computing student's t-interval

Unfortunately, SciPy doesn't support computing confidence intereval of difference in mean separately. It is incorporated into computing t-statistic and p-value of t-test, but users can't access its underlying confidence interval.

But we can computer it easily.

In [3]:
from scipy import stats
import numpy as np

In [4]:
x1 = [12.9, 10.2, 7.4, 7.0, 10.5, 11.9, 7.1, 9.9, 14.4, 11.3]
x2 = [10.2, 6.9, 10.9, 11.0, 10.1, 5.3, 7.5, 10.3, 9.2, 8.8]

alpha = 0.05                                                 # significance level = 5%
n1, n2 = len(x1), len(x2)                                    # sample sizes
s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)              # sample variances
s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2)) # pooled standard deviation
df = n1 + n2 - 2                                             # degrees of freedom
t = stats.t.ppf(1 - alpha/2, df)                             # t-critical value for 95% CI

lower = (np.mean(x1) - np.mean(x2)) - t * np.sqrt(1 / len(x1) + 1 / len(x2)) * s
upper = (np.mean(x1) - np.mean(x2)) + t * np.sqrt(1 / len(x1) + 1 / len(x2)) * s

print(lower, upper)

-0.8520326742900641 3.332032674290068


## Independent (unpaired) samples, unequal variance - Welch's t-interval

When comparing central tendency of normal distributions, it is safer, and therefore recommended to always use Welch's t-test, which assumes unequal variances of samples, as explained below. Equal variance t-test is not robust when population variances are different, but unequal variances are robust even when population variances are equal.

Confidence interval of difference in mean assuming unequal variance (Welch's t-interval) can be calculated as follows:

$$ \text{C.I.}_{\Delta \text{mean}}: \quad (\mu_{1}- \mu_{2}) \pm (t_{1-\frac{\alpha}{2},df} \times \sqrt{\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}}) \tag{5} $$

where

$μ$: sample mean

$α$: significance level

$n$: number of samples

$df$ : degrees of freedom

$s$: sample standard deviation

$t$: t-score. depends on $α$ and degrees of freedom $n−1$


The formula is very similar to student's t-interval. There are two main differences:

* We use each sample's own variance $s_1^2$ and $s_2^2$, instead of pooled (weighted average) variance $s_p^2$.

* Degrees of freedom $df$ is computed with eq (6).

$$ df = \frac{(\frac{s^2_1}{n_1} + \frac{s^2_2}{n_2})^2}{\frac{(s^2_1/n_1)^2}{n_1-1} + \frac{(s^2_2/n_2)^2}{n_2-1}} \tag{6} $$



#### Pythonic Tip: Computing Welch's t-interval

The procedure is very similar to Computing student's t-interval. We will compute confidence interval of difference in mean assuming unequal variance, with eq (5). Although Scipy supports computing t-statistic for Welch's t-test, it doesn't support a function that allows us to compute Welch's t-interval. We will have to write our own codes to compute it. Don't forget to compute sample variance, instead of population variance by setting ddof=1 as explained above.

In [5]:
from scipy import stats
import numpy as np

x1 = [12.9, 10.2, 7.4, 7.0, 10.5, 11.9, 7.1, 9.9, 14.4, 11.3]
x2 = [10.2, 6.9, 10.9, 11.0, 10.1, 5.3, 7.5, 10.3, 9.2, 8.8]

alpha = 0.05                                                       # significance level = 5%
n1, n2 = len(x1), len(x2)                                          # sample sizes
s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)                    # sample variances
df = (s1/n1 + s2/n2)**2 / ((s1/n1)**2/(n1-1) + (s2/n2)**2/(n2-1))  # degrees of freedom
t = stats.t.ppf(1 - alpha/2, df)                                   # t-critical value for 95% CI

lower = (np.mean(x1) - np.mean(x2)) - t * np.sqrt(1 / len(x1) + 1 / len(x2)) * s
upper = (np.mean(x1) - np.mean(x2)) + t * np.sqrt(1 / len(x1) + 1 / len(x2)) * s

print(lower, upper)

-0.8633815129922358 3.3433815129922397


In [ ]:
from scipy import stats
import numpy as np

x1 = [12.9, 10.2, 7.4, 7.0, 10.5, 11.9, 7.1, 9.9, 14.4, 11.3]
x2 = [10.2, 6.9, 10.9, 11.0, 10.1, 5.3, 7.5, 10.3, 9.2, 8.8]

alpha = 0.05                                                       # significance level = 5%
n1, n2 = len(x1), len(x2)                                          # sample sizes
s1, s2 = np.var(x1, ddof=1), np.var(x2, ddof=1)                    # sample variances
df = (s1/n1 + s2/n2)**2 / ((s1/n1)**2/(n1-1) + (s2/n2)**2/(n2-1))  # degrees of freedom
t = stats.t.ppf(1 - alpha/2, df)                                   # t-critical value for 95% CI

lower = (np.mean(x1) - np.mean(x2)) - t * np.sqrt(1 / len(x1) + 1 / len(x2)) * s
upper = (np.mean(x1) - np.mean(x2)) + t * np.sqrt(1 / len(x1) + 1 / len(x2)) * s   #老师这里错了

print(lower, upper)